In [70]:
import numpy as np
import pandas as pd
pd.options.display.max_columns = None

import seaborn as sns
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px

from sklearn import preprocessing
from sklearn.feature_selection import RFECV
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.model_selection import StratifiedKFold, KFold, train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import mean_squared_error, r2_score, f1_score, roc_auc_score
from sklearn.feature_selection import SequentialFeatureSelector as SFS, SelectFromModel
from xgboost import XGBClassifier as XGBC

In [71]:
merged=pd.read_csv('merged.csv')
mergedT=pd.read_csv('mergedTest.csv')

In [72]:
cla_merged=merged.drop(['id','hosp_admittime','hosp_dischtime','icu_intime','icu_outtime','icu_outcome','los_icu'], axis=1)
cla_mergedT=mergedT.drop(['id','hosp_admittime','hosp_dischtime','icu_intime','icu_outtime','icu_outcome','los_icu'],axis=1)


In [73]:
#split X, y

X_train=cla_merged.drop(['icu_death'], axis=1)
y_train=cla_merged['icu_death']

X_test=cla_mergedT.drop(['icu_death'], axis=1)
y_test=cla_mergedT['icu_death']

In [74]:
#normalise
num_cols = X_train.columns[X_train.dtypes.apply(lambda c: np.issubdtype(c, np.number))]
scaler = preprocessing.StandardScaler()
X_train[num_cols] = scaler.fit_transform(X_train[num_cols])

num_cols = X_test.columns[X_test.dtypes.apply(lambda c: np.issubdtype(c, np.number))]
scaler = preprocessing.StandardScaler()
X_test[num_cols] = scaler.fit_transform(X_test[num_cols])

# XGB Classifier

In [75]:
xgbc =XGBC()

xgbc.fit(X_train,y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

In [76]:
thresholds = np.sort(xgbc.feature_importances_)

In [77]:
thresholds

array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.00799079, 0.00884531, 0.00900097, 0.00937437, 0.00947442,
       0.01179871, 0.01222753, 0.01281085, 0.01284778, 0.01293397,
       0.01326057, 0.01341515, 0.01413699, 0.01428767, 0.01436841,
       0.0145807 , 0.01469169, 0.01521369, 0.01524957, 0.0155073 ,
       0.01555718, 0.01555741, 0.01560416, 0.01578127, 0.01587883,
       0.01590443, 0.01639114, 0.01647414, 0.01658215, 0.01662579,
       0.01723813, 0.0174708 , 0.01769689, 0.01776506, 0.01777296,
       0.01801004, 0.01847906, 0.01892657, 0.0190079 , 0.01938342,
       0.01965556, 0.01990213, 0.02117314, 0.02160582, 0.02218368,
       0.02258032, 0.02373662, 0.02554268, 0.02909807, 0.03207846,
       0.03686346, 0.03740731, 0.03855588, 0.05549311], dtype=float32)

In [78]:
for thresh in thresholds:
    selection = SelectFromModel(xgbc, threshold=thresh, prefit=True)
    select_X_train = selection.transform(X_train)
    selection_model = XGBC()
    selection_model.fit(select_X_train, y_train)
    select_X_test = selection.transform(X_test)
    predictions = selection_model.predict(select_X_test)
    accuracy = accuracy_score(y_test, predictions)
    print("Thresh=%.3f, n=%d, Accuracy: %.2f%%" % (thresh, select_X_train.shape[1], accuracy*100.0))

C:\Users\kawai\anaconda3\lib\site-packages\sklearn\base.py:413: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(
C:\Users\kawai\anaconda3\lib\site-packages\sklearn\base.py:413: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(
C:\Users\kawai\anaconda3\lib\site-packages\sklearn\base.py:413: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(


Thresh=0.000, n=59, Accuracy: 90.07%


C:\Users\kawai\anaconda3\lib\site-packages\sklearn\base.py:413: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(
C:\Users\kawai\anaconda3\lib\site-packages\sklearn\base.py:413: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(


Thresh=0.000, n=59, Accuracy: 90.07%


C:\Users\kawai\anaconda3\lib\site-packages\sklearn\base.py:413: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(
C:\Users\kawai\anaconda3\lib\site-packages\sklearn\base.py:413: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(


...

C:\Users\kawai\anaconda3\lib\site-packages\sklearn\base.py:413: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(
C:\Users\kawai\anaconda3\lib\site-packages\sklearn\base.py:413: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(


Thresh=0.032, n=5, Accuracy: 89.80%


C:\Users\kawai\anaconda3\lib\site-packages\sklearn\base.py:413: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(
C:\Users\kawai\anaconda3\lib\site-packages\sklearn\base.py:413: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(
C:\Users\kawai\anaconda3\lib\site-packages\sklearn\base.py:413: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(
C:\Users\kawai\anaconda3\lib\site-packages\sklearn\base.py:413: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(


Thresh=0.037, n=4, Accuracy: 89.95%
Thresh=0.037, n=3, Accuracy: 89.93%
Thresh=0.039, n=2, Accuracy: 89.93%
Thresh=0.055, n=1, Accuracy: 89.93%


C:\Users\kawai\anaconda3\lib\site-packages\sklearn\base.py:413: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(
C:\Users\kawai\anaconda3\lib\site-packages\sklearn\base.py:413: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(
C:\Users\kawai\anaconda3\lib\site-packages\sklearn\base.py:413: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(


In [79]:
selected=pd.DataFrame()
selected['col']=X_train.columns
selected['rank']=xgbc.feature_importances_
selected.sort_values(by='rank', ascending=False,inplace=True)

top24=selected[:24]
selected_feats=top24['col'].to_list()
selected_feats

['ckd',
 'admission_type_SURGICAL SAME DAY ADMISSION',
 'race_encode_Not Specified',
 'aniongap',
 'first_careunit_Neuro Surgical Intensive Care Unit (Neuro SICU)',
 'hypertension',
 'first_careunit_Cardiac Vascular Intensive Care Unit (CVICU)',
 'ihd',
 'creatinine',
 'charlson_score',
 'bicarbonate',
 'height',
 'sodium',
 'bun',
 'pt',
 'first_careunit_Coronary Care Unit (CCU)',
 'first_careunit_Medical/Surgical Intensive Care Unit (MICU/SICU)',
 'stroke',
 'malignant_cancer',
 'wbc',
 'chf',
 'admission_age',
 'atrial_fibrillation',
 'copd']

In [80]:

xgb = XGBC()

parameters = {
    'n_estimators':np.arange(start=2,stop=20,step=2),
    'max_depth':np.arange(start=2,stop=6,step=1),
    'learning_rate':np.arange(start=0.05,stop=0.4,step=0.05)
}
stratifiedCV = StratifiedKFold(n_splits=5)
Best_xgb = GridSearchCV(
    xgb,
    param_grid=parameters,
    scoring='f1',
    cv=stratifiedCV,
    verbose=1,
    n_jobs=-1
)

Best_xgb.fit(X_train[selected_feats],y_train)

Fitting 5 folds for each of 252 candidates, totalling 1260 fits


GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=None, shuffle=False),
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     feature_types=None, gamma=None,
                                     gpu_id=None, grow_policy=None,
                                     importance_...
                                     max_delta_step=None, max_depth=None,
                                     max_leaves=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     n_estimators=100, n_jobs=None,
                                     num_parallel_tree=None, predictor=None,
                                     random_state=None, ...),
             n_jobs=-1,
             param_grid={'learning_rate': array([0.05, 0.1 , 0.15, 0.2 , 0.25, 0.3 , 0.35]),
                         'max_depth': array([2, 3, 4, 5]),
                         'n_estimators': array([ 2,  4,  6,  8, 10, 12, 14, 16, 18])},
             scoring='f1', verbose=1)

In [81]:
Best_xgb.best_estimator_

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.35000000000000003,
              max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=5, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=18, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

In [82]:
Best_xgb.best_score_

0.23237999586219438

In [83]:


xgb_pred = Best_xgb.predict(X_test[selected_feats])
xgb_pp=Best_xgb.predict_proba(X_test[selected_feats])

xgb_posProb = xgb_pp[:,-1]

xgb_f1 = f1_score(y_test, xgb_pred)
xgb_auroc = roc_auc_score(y_test, xgb_pp[:,-1])
print(xgb_f1)
print(xgb_auroc)
pd.crosstab(y_test, xgb_pred)

0.2701688555347092
0.8029396024692979


col_0,0,1
icu_death,,
0,3619,50
1,339,72


# XGB Regressor

In [84]:
reg_merged=merged.drop(['id','hosp_admittime','hosp_dischtime','icu_intime','icu_outtime','icu_outcome'], axis=1)
reg_mergedT=mergedT.drop(['id','hosp_admittime','hosp_dischtime','icu_intime','icu_outtime','icu_outcome'],axis=1)

In [85]:
#split X, y

regX_train=reg_merged.drop(['los_icu'], axis=1)
regy_train=reg_merged['los_icu']

regX_test=reg_mergedT.drop(['los_icu'], axis=1)
regy_test=reg_mergedT['los_icu']

In [86]:
#normalise
num_cols = regX_train.columns[regX_train.dtypes.apply(lambda c: np.issubdtype(c, np.number))]
scaler = preprocessing.StandardScaler()
regX_train[num_cols] = scaler.fit_transform(regX_train[num_cols])

num_cols = regX_test.columns[regX_test.dtypes.apply(lambda c: np.issubdtype(c, np.number))]
scaler = preprocessing.StandardScaler()
regX_test[num_cols] = scaler.fit_transform(regX_test[num_cols])

In [87]:
xgbr =XGBR()

xgbr.fit(regX_train,y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=100, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=None, ...)

In [88]:
thresholds = np.sort(xgbr.feature_importances_)

In [89]:
thresholds

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 1.], dtype=float32)

In [90]:
selected=pd.DataFrame()
selected['col']=regX_train.columns
selected['rank']=xgbr.feature_importances_
selected.sort_values(by='rank', ascending=False,inplace=True)

top=selected[:1]
reg_selected=top['col'].to_list()
reg_selected

['icu_death']

In [92]:
#XGB with forward selected feature

xgbr = XGBR()

parameters = {
    'n_estimators':np.arange(start=2,stop=20,step=2),
    'max_depth':np.arange(start=2,stop=6,step=1),
    'learning_rate':np.arange(start=0.05,stop=0.4,step=0.05)
}
kfoldCV = KFold(n_splits=5)
Best_xgbr = GridSearchCV(
    xgbr,
    param_grid=parameters,
    cv=kfoldCV,
)

Best_xgbr.fit(regX_train[reg_selected],regy_train)

GridSearchCV(cv=KFold(n_splits=5, random_state=None, shuffle=False),
             estimator=XGBRegressor(base_score=None, booster=None,
                                    callbacks=None, colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None,
                                    early_stopping_rounds=None,
                                    enable_categorical=False, eval_metric=None,
                                    feature_types=None, gamma=None, gpu_id=None,
                                    grow_policy=None, importance_type=None,
                                    i...
                                    max_cat_to_onehot=None, max_delta_step=None,
                                    max_depth=None, max_leaves=None,
                                    min_child_weight=None, missing=nan,
                                    monotone_constraints=None, n_estimators=100,
                                    n_jobs=None, num_parallel_tree=None,
                                    predictor=None, random_state=None, ...),
             param_grid={'learning_rate': array([0.05, 0.1 , 0.15, 0.2 , 0.25, 0.3 , 0.35]),
                         'max_depth': array([2, 3, 4, 5]),
                         'n_estimators': array([ 2,  4,  6,  8, 10, 12, 14, 16, 18])})

In [93]:
Best_xgbr.best_estimator_

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.35000000000000003,
             max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=2, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=14, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=None, ...)

In [95]:
xgbr_pred = Best_xgbr.predict(regX_test[reg_selected])
xgbr_r2 = r2_score(regy_test, xgbr_pred)

xgbr_r2

0.008997742490554295

# regressor using classifier features

In [96]:
#XGB with forward selected feature

xgbr = XGBR()

parameters = {
    'n_estimators':np.arange(start=2,stop=20,step=2),
    'max_depth':np.arange(start=2,stop=6,step=1),
    'learning_rate':np.arange(start=0.05,stop=0.4,step=0.05)
}
kfoldCV = KFold(n_splits=5)
Best_xgbr = GridSearchCV(
    xgbr,
    param_grid=parameters,
    cv=kfoldCV,
)

Best_xgbr.fit(regX_train[selected_feats],regy_train)

GridSearchCV(cv=KFold(n_splits=5, random_state=None, shuffle=False),
             estimator=XGBRegressor(base_score=None, booster=None,
                                    callbacks=None, colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None,
                                    early_stopping_rounds=None,
                                    enable_categorical=False, eval_metric=None,
                                    feature_types=None, gamma=None, gpu_id=None,
                                    grow_policy=None, importance_type=None,
                                    i...
                                    max_cat_to_onehot=None, max_delta_step=None,
                                    max_depth=None, max_leaves=None,
                                    min_child_weight=None, missing=nan,
                                    monotone_constraints=None, n_estimators=100,
                                    n_jobs=None, num_parallel_tree=None,
                                    predictor=None, random_state=None, ...),
             param_grid={'learning_rate': array([0.05, 0.1 , 0.15, 0.2 , 0.25, 0.3 , 0.35]),
                         'max_depth': array([2, 3, 4, 5]),
                         'n_estimators': array([ 2,  4,  6,  8, 10, 12, 14, 16, 18])})

In [97]:
Best_xgbr.best_estimator_

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.35000000000000003,
             max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=3, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=18, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=None, ...)

In [98]:
xgbr_pred = Best_xgbr.predict(regX_test[selected_feats])
xgbr_r2 = r2_score(regy_test, xgbr_pred)

xgbr_r2

0.08792965118208629